In [ ]:
import imutils
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy as np

In [ ]:
import cv2
import numpy as np

def preprocess(refImg, disImg):
    refImg = cv2.cvtColor(refImg, cv2.COLOR_BGR2GRAY)
    disImg = cv2.cvtColor(disImg, cv2.COLOR_BGR2GRAY)
    return refImg.astype(np.float64), disImg.astype(np.float64)

def getDirectionalGradient(f):
    kernel1 = np.zeros((5, 5))
    Kt = (1 / 16) * np.array([[3, 10, 3], [0, 0, 0], [-3, -10, -3]])
    kernel1[1:4, 1:4] = Kt
    kernel2 = kernel1.T
    kernel3 = (1 / 16) * np.array([[0, 0, 3, 0, 0],
                                   [0, 10, 0, 0, 0],
                                   [3, 0, 0, 0, -3],
                                   [0, 0, 0, -10, 0],
                                   [0, 0, -3, 0, 0]])
    kernel4 = np.rot90(kernel3)

    g = np.zeros((f.shape[0], f.shape[1], 4))
    g[:, :, 0] = cv2.filter2D(f, -1, kernel1, borderType=cv2.BORDER_REFLECT)
    g[:, :, 1] = cv2.filter2D(f, -1, kernel2, borderType=cv2.BORDER_REFLECT)
    g[:, :, 2] = cv2.filter2D(f, -1, kernel3, borderType=cv2.BORDER_REFLECT)
    g[:, :, 3] = cv2.filter2D(f, -1, kernel4, borderType=cv2.BORDER_REFLECT)
    return g


def SG_ESSIM(refImg, disImg):
    refImg, disImg = preprocess(refImg, disImg)
    refRows, refCols = refImg.shape[:2]
    disRows, disCols = disImg.shape[:2]

    gRef = getDirectionalGradient(refImg)
    gDis = getDirectionalGradient(disImg)

    params = getParams()

    gradRef = np.abs(gRef[:, :, [0, 2]] - gRef[:, :, [1, 3]]) ** 0.5
    gradDis = np.abs(gDis[:, :, [0, 2]] - gDis[:, :, [1, 3]]) ** 0.5

    Y, X = np.meshgrid(np.arange(1, min(refCols, disCols) + 1), np.arange(1, min(refRows, disRows) + 1))
    ind3 = np.argmax(gradRef, axis=2)

    Y_ind, X_ind = np.unravel_index(ind3.ravel(), gradRef.shape[:2])
    ind = np.ravel_multi_index((Y_ind, X_ind), gradRef.shape[:2])
    edgeMap = np.maximum(gradRef.ravel()[ind], gradDis.ravel()[ind])

    H = params['C'] * np.exp(-edgeMap / params['h'])
    SM = (2 * gradRef.ravel()[ind] * gradDis.ravel()[ind] + H) / (gradRef.ravel()[ind] ** 2 + gradDis.ravel()[ind] ** 2 + H)
    quality = np.mean(SM)

    return quality


def getParams():
    params = {'h': 0.5, 'L': 255, 'K': 200}
    params['C'] = (params['K'] * params['L']) ** (2 ** 0.5)
    return params

# Example usage
refImg = cv2.imread('test/Reference_.HDR')
disImg = refImg

quality = SG_ESSIM(refImg, disImg)
print('SG-ESSIM Quality:', quality)

In [ ]:
def scale(img,pixel_x):
    return cv2.resize(img, (pixel_x, pixel_x))

def draw_quadrilateral(image, points):
    pts = np.array(points, np.int32)
    pts = pts.reshape((-1, 1, 2))
    
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    
    cv2.fillPoly(mask, [pts], 255)
    mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2) 
    result = np.where(mask, image, 0)
    return result


In [86]:
def write_sources(source_file, ref_file, points):
    # read images from their path
    base_name = os.path.basename(source_file)
    source = cv2.imread(source_file)
    ref = cv2.imread(ref_file)

    # crop source images and find corner points of the window
    num_dec = 44
    x_list = [i[1] for i in points]
    y_list = [i[0] for i in points]
    min_x, max_x = min(x_list), max(x_list)
    min_y, max_y = min(y_list), max(y_list)
    decrease_source = source[num_dec:-num_dec, num_dec:-num_dec, :]
    
    # scale images and draw quadrilateral of window for each of them
    scaled_source = scale(decrease_source, 600)
    scaled_ref = scale(ref, 600)
    quad_source = draw_quadrilateral(scaled_source, points)[min_x:max_x, min_y:max_y, :]
    quad_ref = draw_quadrilateral(scaled_ref, points )[min_x:max_x, min_y:max_y, :]
    
    # convert images to grayscle and get the edges of them(canny)
    gray_source = cv2.cvtColor(quad_source, cv2.COLOR_BGR2GRAY)    
    gray_ref = cv2.cvtColor(quad_ref, cv2.COLOR_BGR2GRAY)
    edges_source = cv2.Canny(gray_source, 100, 200)
    edges_ref = cv2.Canny(gray_ref, 100, 200)
    
    # resize refrence and sample images and get sg_essim of them
    rs1, rs2 = cv2.resize(quad_source, (200, 200)), cv2.resize(quad_ref, (200, 200))
    sg_essim_score = SG_ESSIM(rs1, rs2)
    
    # define a new array and mask it out of the window
    gray_ref_3d = np.repeat(gray_ref[:, :, np.newaxis], quad_ref.shape[2], axis=2)
    masked_ref_sg = np.where(gray_ref_3d < 100, quad_ref, 255)
    masked_source_sg = np.where(gray_ref_3d < 100, quad_source, 255)
    
    # get the result for masked images    
    rs3, rs4 = cv2.resize(masked_ref_sg, (200, 200)), cv2.resize(masked_source_sg, (200, 200))
    sg_c_crop_score = SG_ESSIM(rs3, rs4)

    return base_name, sg_c_crop_score, sg_essim_score
    